In [1]:
pip install fuzzywuzzy[speedup] pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import folium
from folium import Marker
from folium.plugins import HeatMap
from fuzzywuzzy import process

In [3]:
# Column names for data
uni_data = pd.DataFrame(columns=['University Name','In State Tuition','Out of State Tuition','Admission Rate','Number of Students','Graduation Rate 4-years',
                                'Earnings After 10 Years','Retention Rate','Diversity Score','Public or Private'])

# List of university to scrape
university_list = ['Princeton University','Massachusetts Institute of Technology','Harvard University','Stanford University','Yale University',
                   'California Institute of Technology','Duke University','Johns Hopkins University','Northwestern University',
                   'University of Pennsylvania','Cornell University','University of Chicago','Brown University','Columbia University',
                   'Dartmouth College','University of California-Los Angeles','University of California-Berkeley','Rice University','University of Notre Dame','Vanderbilt University', 'Carnegie Mellon University',
                  'University of Michigan','Washington University','Emory University',"Georgetown University","University of Virginia",
                   "University of North Carolina-Chapel Hill", "University of Southern California","University of California-San Diego",
                   "New York University","University of Florida","University of Texas-Austin","Georgia Institute of Technology",
                   "University of California-Davis","University of California-Irvine","University of Illinois-Urbana-Champaign","Boston College",
                   "Tufts University","University of California-Santa Barbara","University of Wisconsin-Madison","Boston University",
                   "Ohio State University","Rutgers University","University of Maryland","University of Rochester",
                   "Lehigh University","Purdue University","University of Georgia","University of Washington","Wake Forest University"]

# Loops through list of universities
for university in university_list:
    # Obtains the data from the API
    url = "https://api.data.gov/ed/collegescorecard/v1/schools?api_key=OvEQIo28yxxsUWagGnOrXt9vuxU1WbanzSm1V8Dp&school.name="+university+"&fields=id,school.name,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,latest.admissions.admission_rate.overall,latest.student.size,latest.completion.rate_suppressed.four_year,latest.earnings.10_yrs_after_entry.median,latest.student.retention_rate.overall.full_time,latest.school.peps_ownership,latest.student.demographics.race_ethnicity.asian,latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.white"
    response = requests.get(url)
    data = response.json()

    # Gets each metric from the data dictionary 
    in_state_tuition = data['results'][0]['latest.cost.tuition.in_state']
    out_of_state_tuition = data['results'][0]['latest.cost.tuition.out_of_state']
    admission_rate = data['results'][0]['latest.admissions.admission_rate.overall']
    student_size = data['results'][0]['latest.student.size']
    completion_rate = data['results'][0]['latest.completion.rate_suppressed.four_year']
    earning_10year = data['results'][0]['latest.earnings.10_yrs_after_entry.median']
    retention_rate = data['results'][0]['latest.student.retention_rate.overall.full_time']
    ownership = data['results'][0]['latest.school.peps_ownership']
    prop_white = data['results'][0]['latest.student.demographics.race_ethnicity.white']
    prop_black = data['results'][0]['latest.student.demographics.race_ethnicity.black']
    prop_asian = data['results'][0]['latest.student.demographics.race_ethnicity.asian']

    # Calculates the diversity score from the ethic demographics
    if all(p is not None for p in [prop_white, prop_black, prop_asian]):
        prop_other = 1-prop_white-prop_black-prop_asian
        proportions = [prop_white,prop_black,prop_asian,prop_other]
        diversity_score = 1 - sum(p**2 for p in proportions)
    else:
        diversity_score = None

    # Adds the collected data from the API for the university to the dataframe
    uni_data.loc[len(uni_data)] = {'University Name': university, 'In State Tuition': in_state_tuition, 'Out of State Tuition': out_of_state_tuition, 
                                   'Admission Rate': admission_rate, 'Number of Students': student_size,'Graduation Rate 4-years': completion_rate,
                                  'Earnings After 10 Years': earning_10year,'Retention Rate': retention_rate, 'Diversity Score': diversity_score,
                                   'Public or Private': ownership}

In [4]:
# Inputs missing data for cornell university
uni_data.loc[10, 'Admission Rate'] = 0.079
uni_data.loc[10, 'Number of Students'] = 16071
uni_data.loc[10, 'In State Tuition'] = 63200
uni_data.loc[10, 'Out of State Tuition'] = 63200
uni_data.loc[10, 'Retention Rate'] = 0.96
1-0.385-0.077-0.363
proportions = [0.303,0.0588,0.179,1-0.303-0.0588-0.179]
diversity_score = 1 - sum(p**2 for p in proportions)
uni_data.loc[10, 'Diversity Score'] = diversity_score
# Inputs missing data for California Institute of Technology
uni_data.loc[5, 'Admission Rate'] = 0.027
# Inputs missing data for Boston College
uni_data.loc[36, 'Admission Rate'] = 0.167
# Inputs missing data for Ohio State University
uni_data.loc[41, 'Admission Rate'] = 0.027

In [5]:
# Displays data
uni_data.head(50)

,University Name,In State Tuition,Out of State Tuition,Admission Rate,Number of Students,Graduation Rate 4-years,Earnings After 10 Years,Retention Rate,Diversity Score,Public or Private
0,Princeton University,57410,57410,0.0570,5527,0.9744,110066,0.9705,0.701479,Private Nonprofit
1,Massachusetts Institute of Technology,57986,57986,0.0396,4638,0.9557,143372,0.9898,0.699191,Private Nonprofit
2,Harvard University,57261,57261,0.0324,7973,0.9726,101817,0.9769,0.703174,Private Nonprofit
3,Stanford University,58416,58416,0.0368,7761,0.9534,124080,0.9831,0.694972,Private Nonprofit
4,Yale University,62250,62250,0.0457,6639,0.9736,100533,0.9810,0.702715,Private Nonprofit
5,California Institute of Technology,19190,19190,0.0270,540,0.7828,53598,0.9474,0.483280,Proprietary
6,Duke University,62688,62688,0.0635,6570,0.9625,97800,0.9685,0.700444,Private Nonprofit
7,Johns Hopkins University,60480,60480,0.0725,5643,0.9433,87555,0.9695,0.688898,Private Nonprofit
8,Northwestern University,8672,19460,0.9793,6243,0.4528,47021,0.6115,0.600800,Public
9,University of Pennsylvania,63452,63452,0.0650,10572,0.9615,111371,0.9837,0.709739,Private Nonprofit


In [ ]:
# Uploads data to csv
uni_data.to_csv("scorecard_data", index=False)